In [1]:
import pyrpio
from pyrpio import mdio

# Configure options
#options = pyrpio.RPIOOptions(
options = pyrpio.RPIOConfigs(
  gpiomem=True
)
# Must be called prior to using any interface
pyrpio.configure(options)

### MDIO Operations ###

# Create bus using GPIO_PIN 23 [clk] and 24 [data](bit-bang)
# GPIO_PIN no. and pin no. are different. Use command "pinout" in RP to check it.
# pin 6 / 20 / 25 are all GND
mdio_bus = mdio.MDIO(23, 24)
mdio_bus.open()

0

In [4]:
def stat(pid):
    phy_LS = (mdio_bus.read_c22_register(pid, 1) & 0x4) >> 2
    TPI = mdio_bus.read_c22_register(pid, 24)
    sgmii_stat = mdio_bus.read_c45_register(pid, 30, 9)
    sgmii_LS = (sgmii_stat & 4) >> 2
    sgmii_Speed = sgmii_stat & 3
    fw = mdio_bus.read_c22_register(pid, 30)
    print("PHY ID: ", pid)
    print("FW version: 0x%04x"%fw)
    LS_list = ["Down", "Up"]
    LS_speed_list = ["10Mbps", "100Mbps", "1000Mbps", "Autonegotiation mode", "2.5Gbps"]
    print ("PHY_LS: ", LS_list[phy_LS], " -- Speed: ", LS_speed_list[TPI&0x7])
    sgmii_speed_list = ["10Mbps", "100Mbps", "1Gbps", "2.5Gbps"]
    print("SGMII LS: ", LS_list[sgmii_LS], " -- Speed: ", sgmii_speed_list[sgmii_Speed])
    print("")
    return

In [5]:
stat(0x1c)
stat(0x1d)
stat(0x1e)
stat(0x1f)

PHY ID:  28
FW version: 0x8959
PHY_LS:  Down  -- Speed:  Autonegotiation mode
SGMII LS:  Down  -- Speed:  10Mbps

PHY ID:  29
FW version: 0x8959
PHY_LS:  Down  -- Speed:  Autonegotiation mode
SGMII LS:  Down  -- Speed:  10Mbps

PHY ID:  30
FW version: 0x8959
PHY_LS:  Down  -- Speed:  Autonegotiation mode
SGMII LS:  Down  -- Speed:  10Mbps

PHY ID:  31
FW version: 0x8959
PHY_LS:  Down  -- Speed:  Autonegotiation mode
SGMII LS:  Down  -- Speed:  10Mbps



# Read register 30.14 for Junc. temperature

In [29]:
cal = [ [0, -52], [3, -51], [6, -50], [9, -49],[12, -48], [16, -47], [19, -46], [22, -45], \
        [26, -44], [29, -43], [33, -42], [36, -41],[39, -40], [43, -39], [46, -38], [50, -37],\
		[53, -36], [57, -35], [60, -34], [64, -33],
		[67, -32], [71, -31], [74, -30], [78, -29],
		[81, -28], [85, -27], [89, -26], [92, -25],
		[96, -24], [99, -23], [103, -22], [107, -21],
		[111, -20], [114, -19], [118, -18], [122, -17],
		[126, -16], [129, -15], [133, -14], [137, -13],
		[141, -12], [145, -11], [149, -10], [152, -9],
		[156, -8], [160, -7], [164, -6], [168, -5],
		[172, -4], [176, -3], [180, -2], [184, -1],
		[188, 0], [193, 1], [197, 2], [201, 3],
		[205, 4], [209, 5], [213, 6], [217, 7],
		[222, 8], [226, 9], [230, 10], [234, 11],
		[239, 12], [243, 13], [247, 14], [252, 15],
		[256, 16], [261, 17], [265, 18], [270, 19],
		[274, 20], [279, 21], [283, 22], [288, 23],
		[292, 24], [297, 25], [301, 26], [306, 27],
		[311, 28], [315, 29], [320, 30], [325, 31],
		[330, 32], [334, 33], [339, 34], [344, 35],
		[349, 36], [354, 37], [359, 38], [364, 39],
		[369, 40], [374, 41], [379, 42], [384, 43],
		[389, 44], [394, 45], [399, 46], [404, 47],
		[409, 48], [414, 49], [420, 50], [425, 51],
		[430, 52], [436, 53], [441, 54], [446, 55],
		[452, 56], [457, 57], [463, 58], [468, 59],
		[474, 60], [479, 61], [485, 62], [490, 63],
		[496, 64], [502, 65], [507, 66], [513, 67],
		[519, 68], [525, 69], [530, 70], [536, 71],
		[542, 72], [548, 73], [554, 74], [560, 75],
		[566, 76], [572, 77], [578, 78], [584, 79],
		[591, 80], [597, 81], [603, 82], [609, 83],
		[616, 84], [622, 85], [628, 86], [635, 87],
		[641, 88], [648, 89], [654, 90], [661, 91],
		[667, 92], [674, 93], [680, 94], [687, 95],
		[694, 96], [701, 97], [708, 98], [714, 99],
		[721, 100], [728, 101], [735, 102], [742, 103],
		[749, 104], [756, 105], [764, 106], [771, 107],
		[778, 108], [785, 109], [793, 110], [800, 111],
		[807, 112], [815, 113], [822, 114], [830, 115],
		[838, 116], [845, 117], [853, 118], [861, 119],
		[869, 120], [876, 121], [884, 122], [892, 123],
		[900, 124], [908, 125], [917, 126], [925, 127],
		[933, 128], [941, 129], [950, 130], [958, 131],
		[967, 132], [975, 133], [984, 134], [993, 135],
		[1002, 136], [1010, 137], [1019, 138],
	]

def temp(phyid):
    # Read register 30.14 from phy id 0x1c-0x1f (CLAUSE-45)
    d = mdio_bus.read_c45_register(phyid, 30, 14)
    #print ("0x%04x"%d, d)

    a = -53
    for i in cal:
        if d < i[0]:
            T = a
            break
        else:
            a = i[1]
    print ("Phy_id:", phyid, "  Junc. Temp =", T)
    return T
        

In [30]:
temp(0x1c)
temp(0x1d)
temp(0x1e)
temp(0x1f)


Phy_id: 28   Junc. Temp = 54
Phy_id: 29   Junc. Temp = 54
Phy_id: 30   Junc. Temp = 54
Phy_id: 31   Junc. Temp = 54


54

## Disable 2500Base-T capability in ANEG

In [8]:
def disable_2G5BT(pid):
    # Read MMD 7.32, default value = 0x40a2
    print ("phy_id:", pid)
    d = mdio_bus.read_c45_register(pid, 7, 32)
    print ("Before: 0x%04x"%d)
    # Write MMD 7.32 = 0x4002 to disable 2.5GBASE-T capability
    mdio_bus.write_c45_register(pid, 7, 32, 0x4002)
    d = mdio_bus.read_c45_register(pid, 7, 32)
    print ("After: 0x%04x"%d)
    return

disable_2G5BT(0x1c)
disable_2G5BT(0x1d)
disable_2G5BT(0x1e)
disable_2G5BT(0x1f)


phy_id: 28
Before: 0x40a2
After: 0x4002
phy_id: 29
Before: 0x40a2
After: 0x4002
phy_id: 30
Before: 0x40a2
After: 0x4002
phy_id: 31
Before: 0x40a2
After: 0x4002


## LED configuration

In [19]:
# Read PHY_LED 0.27
def rd_phy_led(pid):
    d = mdio_bus.read_c22_register(pid, 27)
    print ("0x%04x,"%d, " phy_id: 0x%02x"%pid)
rd_phy_led(0x1c)
rd_phy_led(0x1d)
rd_phy_led(0x1e)
rd_phy_led(0x1f)


0xff00,  phy_id: 0x1c
0xff00,  phy_id: 0x1d
0xfe00,  phy_id: 0x1e
0xff00,  phy_id: 0x1f


In [24]:
# Read VSPEC1_LED0-2 (slow blink, pulse, constant on, fast blink)
def rd_vspec_led(pid):
    d = mdio_bus.read_c45_register(pid, 0x1e, 1)
    e = mdio_bus.read_c45_register(pid, 0x1e, 2)
    f = mdio_bus.read_c45_register(pid, 0x1e, 3)
    print("phy_id: 0x%02x, "%pid, "LED0 = 0x%04x,"%d, "LED1 = 0x%04x,"%e, "LED2 = 0x%04x,"%f, )

rd_vspec_led(0x1c)
rd_vspec_led(0x1d)
rd_vspec_led(0x1e)
rd_vspec_led(0x1f)


phy_id: 0x1c,  LED0 = 0x0330, LED1 = 0x0380, LED2 = 0x0340,
phy_id: 0x1d,  LED0 = 0x0330, LED1 = 0x0380, LED2 = 0x0340,
phy_id: 0x1e,  LED0 = 0x0330, LED1 = 0x0380, LED2 = 0x0340,
phy_id: 0x1f,  LED0 = 0x0330, LED1 = 0x0380, LED2 = 0x0340,


In [28]:
mdio_bus.write_c22_register(0x1e, 27, 0xf700)

0

## Close pyMDIO master

In [4]:
# Close up shop
mdio_bus.close()

0

In [6]:
d = mdio_bus.read_c22_register(0x1c, 2)
print ("0x%04x"%d)
d = mdio_bus.read_c22_register(0x1d, 2)
print ("0x%04x"%d)
d = mdio_bus.read_c22_register(0x1e, 2)
print ("0x%04x"%d)
d = mdio_bus.read_c22_register(0x1f, 2)
print ("0x%04x"%d)

0x67c9
0x67c9
0x67c9
0x67c9


In [14]:
mdio_bus.write_c22_register(0, 25, 0x55aa)
d = mdio_bus.read_c22_register(0,25)
print ("0x%04x"%d)

0x55aa


In [15]:
mdio_bus.write_c45_register(0,0, 25, 0xcc33)
d = mdio_bus.read_c45_register(0,0,25)
print ("0x%04x"%d)

0xcc33


In [11]:
# Read register set from device 0x30 (CLAUSE-45)
mdio_bus.read_c45_registers(0x1c, 30, [8,1,2,3,4])

[12482, 816, 896, 832, 896]

## PDI (XPCS ...) access
#### USXGMII status and configuration

In [6]:
import time

def PHY_wr_mmd(p, a, reg, d):
    print ("port set phy-mmd-reg port 0 mmd-addr", a, "mmd-reg", reg>>1, "data 0x%04x"%d)
    mdio_bus.write_c45_register(p, a, reg>>1, d)
    return
def PHY_rd_mmd(p,a,reg):
    ret = mdio_bus.read_c45_register(p,a,reg>>1)
    print ("port get phy-mmd-reg port 0 mmd-addr", a, "mmd-reg", reg>>1)
    return ret

def gpy2xx_mbox_write32(pid, addr, data, pr = 0):
    dl = data & 0xffff
    PHY_wr_mmd(pid, 30, 10, dl) # data low, VSPEC1_MBOX_DATA
    PHY_wr_mmd(pid, 30, 26, data>>16) # data high, VSPEC1_MBOX_DATA_HIGH
    PHY_wr_mmd(pid, 30, 12, addr & 0xFFFF) # addr low, VSPEC1_MBOX_ADDRLO
    cmd = ((addr >> 16) & 0xff) + (5<<8)
    #cmd_status = PHY_rd_mmd(pid, 30, 14)  # bit15: ready, bit14: fail
    if pr == 1:
        print ("wr32: pid =", pid, "addr = 0x%04x"%addr, "data = 0x%04x"%data, "mbox_cmd = 0x%04x"%cmd, " : mbox status before write32 = 0x%04x"%cmd_status)
    PHY_wr_mmd(pid, 30, 14, cmd) # 
    time.sleep(0.1)
    return
    
def gpy2xx_mbox_write16(pid, addr, data, pr = 0):
    PHY_wr_mmd(pid, 30, 10, data & 0xFFFF) # data low, VSPEC1_MBOX_DATA
    PHY_wr_mmd(pid, 30, 12, addr & 0xFFFF) # addr low, VSPEC1_MBOX_ADDRLO
    cmd = ((addr >> 16) & 0xff) + (1<<8)
    #cmd_status = PHY_rd_mmd(pid, 30, 14)  # bit15: ready, bit14: fail
    if pr == 1:
        print ("wr16: pid =", pid, "addr = 0x%04x"%addr, "data = 0x%04x"%data, "mbox_cmd = 0x%04x"%cmd, " : mbox status before write16 = 0x%04x"%cmd_status)
    PHY_wr_mmd(pid, 30, 14, cmd) # 
    time.sleep(0.1)
    return
    
def gpy2xx_mbox_read16(pid, addr, pr = 0):
    PHY_wr_mmd(pid, 30, 12, addr & 0xFFFF) # addr low, VSPEC1_MBOX_ADDRLO
    cmd = ((addr >> 16) & 0xff) + (0<<8)
    #cmd_status = PHY_rd_mmd(pid, 30, 14)  # bit15: ready, bit14: fail
    if pr == 1:
        print ("rd16: pid =", pid, "addr = 0x%04x"%addr, "mbox_cmd = 0x%04x"%cmd, " : mbox status before read16 = 0x%04x"%cmd_status)
    PHY_wr_mmd(pid, 30, 14, cmd) # 
    time.sleep(0.1)
    #cmd_status = PHY_rd_mmd(pid, 30, 14)  # bit15: ready, bit14: fail
    ret = PHY_rd_mmd(pid, 30, 10)
    if pr == 1:
        print ("rd16: read data = 0x%04x"%ret, "mbox status = 0x%04x"%cmd_status)
    return ret
    
def PHY_XPCS_HWRD(pid, xpcs_addr, pr = 0):
    gpy2xx_mbox_write32(pid, 0x00D3DFFC, xpcs_addr >> 10, pr)
    ret = gpy2xx_mbox_read16(pid, 0x00D3DC00 + (xpcs_addr & 0x3ff), pr)
    return ret

def PHY_XPCS_HWWR(pid, xpcs_addr, data, pr = 0):
    gpy2xx_mbox_write32(pid, 0x00D3DFFC, xpcs_addr >> 10, pr)
    gpy2xx_mbox_write16(pid, 0x00D3DC00 + (xpcs_addr & 0x3ff), data, pr)
    

In [7]:
XPCS_VS_MII_MMD_XPCS_SR_MII_STS = 0x007C0004
mii_status = PHY_XPCS_HWRD(0x1c, XPCS_VS_MII_MMD_XPCS_SR_MII_STS, 0)
print ("mii_status: 0x%04x"%mii_status, "---- 0x007C0004 : XPCS_VS_MII_MMD_XPCS_SR_MII_STS")

port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5 data 0x1f00
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 13 data 0x0000
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdffc
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x05d3
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdc04
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x00d3
port get phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5
mii_status: 0x010d ---- 0x007C0004 : XPCS_VS_MII_MMD_XPCS_SR_MII_STS


In [10]:
XPCS_PMA_MMD_XPCS_SR_PMA_EXT_ABL = 0x0004002C
mii_mode = PHY_XPCS_HWRD(0x1c, XPCS_PMA_MMD_XPCS_SR_PMA_EXT_ABL, 0)
print ("mii_mode: 0x%04x"%mii_mode, "---- 0x0004002C : XPCS_PMA_MMD_XPCS_SR_PMA_EXT_ABL")
# 0x0050 = usxgmii

port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5 data 0x0100
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 13 data 0x0000
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdffc
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x05d3
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdc2c
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x00d3
port get phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5
mii_mode: 0x0050 ---- 0x0004002C : XPCS_PMA_MMD_XPCS_SR_PMA_EXT_ABL


In [13]:
XPCS_PMA_MMD_XPCS_SR_PMA_STATUS1 = 0x00040004
pma_rlu = PHY_XPCS_HWRD(0x1c, XPCS_PMA_MMD_XPCS_SR_PMA_STATUS1, 0)
print ("Read PMA RLU: 0x%04x"%pma_rlu, "---- 0x00040004 : XPCS_PMA_MMD_XPCS_SR_PMA_STATUS1")
pma_rlu = PHY_XPCS_HWRD(0x1c, XPCS_PMA_MMD_XPCS_SR_PMA_STATUS1, 0)
print ("Read PMA RLU: 0x%04x"%pma_rlu, "---- 0x00040004 : XPCS_PMA_MMD_XPCS_SR_PMA_STATUS1")

port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5 data 0x0100
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 13 data 0x0000
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdffc
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x05d3
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdc04
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x00d3
port get phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5
Read PMA RLU: 0x0006 ---- 0x00040004 : XPCS_PMA_MMD_XPCS_SR_PMA_STATUS1


In [14]:
XPCS_XS_PCS_MMD_XPCS_SR_XS_PCS_STS1 = 0x000c0004
pcs_rlu = PHY_XPCS_HWRD(0x1c, XPCS_XS_PCS_MMD_XPCS_SR_XS_PCS_STS1, 0)
print ("Read PCS RLU: 0x%04x"%pcs_rlu, "---- 0x000c0004 : XPCS_XS_PCS_MMD_XPCS_SR_XS_PCS_STS1")
pcs_rlu = PHY_XPCS_HWRD(0x1c, XPCS_XS_PCS_MMD_XPCS_SR_XS_PCS_STS1, 0)
print ("Read PCS RLU: 0x%04x"%pcs_rlu, "---- 0x000c0004 : XPCS_XS_PCS_MMD_XPCS_SR_XS_PCS_STS1")


port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5 data 0x0300
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 13 data 0x0000
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdffc
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x05d3
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdc04
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x00d3
port get phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5
Read PCS RLU: 0x0042 ---- 0x000c0004 : XPCS_XS_PCS_MMD_XPCS_SR_XS_PCS_STS1
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5 data 0x0300
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 13 data 0x0000
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdffc
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x05d3
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdc04
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x00d3
port get phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5
Read PCS RLU: 0x0046 ---- 0x000c0004 : XPCS_XS_PCS_MMD_XPCS_SR_XS_PCS_STS1


## Smartbyte (Paiteng S6800-8TH-4XF) 

#### Read USXGMII status
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5 data 0x1f00  
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 13 data 0x0000  
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdffc  
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x05d3  
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdc04  
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x00d3  
port get phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5  

port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 5 data 0x1f00  
port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 13 data 0x0000  
port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 6 data 0xdffc  
port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 7 data 0x05d3  
port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 6 data 0xdc04  
port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 7 data 0x00d3  
port get phy-mmd-reg port 4 mmd-addr 30 mmd-reg 5  

#### Write USXGMII PCS SW Reset
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5 data 0x0380  
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 13 data 0x0000  
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdffc  
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x05d3  
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5 data 0xa200  
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdc00  
port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x05d3  

port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 5 data 0x0380  
port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 13 data 0x0000  
port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 6 data 0xdffc  
port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 7 data 0x05d3  
port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 5 data 0xa200  
port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 6 data 0xdc00  
port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 7 data 0x05d3  

    
    
    
    RTK.0>   
RTK.0>  
RTK.0>  
RTK.0� port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5 data 0x1f00  
RTK.0> port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 13 data 0x0000  
RTK.0� port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdffc  
RTK.0> port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x05d3  
RTK.0> port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdc04  
RTK.0> port set phy-mmd-reg prt 0 mmd-addr 30 mmd-reg 7 data 0x00d3  
RTK.0> port get phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5  
  
Port : 0  
    MMD Addr 30 :   
        0x05          0x0109    
RTK.0>   
RTK.0>   
RTK.0> port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 5 data 0x1f00  
RTK.0� port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 13 data 0x0000  
RTK.0� port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 6 data 0xdffc   
RTK.0> port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 7 data 0x05d3  
RTK.0> port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 6 data 0xdc04  
RTK.0> port set phy-mmd-reg port 4 mmd-addr 30 mmd-reg 7 data 0x00d3  
RTK.0> port get phy-mmd-reg port 4 mmd-addr 30 mmd-reg 5  
  
Port : 4  
    MMD Addr 30 :   
        0x05          0x0109    
RTK.0> port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5 data 0x1f00  
RTK.0> port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 13 data 0x0000  
RTK.0> port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdffc  
RTK.0> port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x05d3  
RTK.0> port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 6 data 0xdc04  
RTK.0> port set phy-mmd-reg port 0 mmd-addr 30 mmd-reg 7 data 0x00d3  
RTK.0> port get phy-mmd-reg port 0 mmd-addr 30 mmd-reg 5  
  
Port : 0  
    MMD Addr 30 :   
        0x05          0x010D    
  

In [34]:
!pinout


,--------------------------------.
| oooooooooooooooooooo J8   +======
| 1ooooooooooooooooooo  PoE |   Net
|  Wi                    oo +======
|  Fi  Pi Model 4B  V1.2 oo      |
|        ,----.               +====
| |D|    |SoC |               |USB3
| |S|    |    |               +====
| |I|    `----'                  |
|                   |C|       +====
|                   |S|       |USB2
| pwr   |HD|   |HD| |I||A|    +====
`-| |---|MI|---|MI|----|V|-------'

Revision           : b03112
SoC                : BCM2711
RAM                : 2048Mb
Storage            : MicroSD
USB ports          : 4 (excluding power)
Ethernet ports     : 1
Wi-fi              : True
Bluetooth          : True
Camera ports (CSI) : 1
Display ports (DSI): 1

J8:
   3V3  (1) (2)  5V    
 GPIO2  (3) (4)  5V    
 GPIO3  (5) (6)  GND   
 GPIO4  (7) (8)  GPIO14
   GND  (9) (10) GPIO15
GPIO17 (11) (12) GPIO18
GPIO27 (13) (14) GND   
GPIO22 (15) (16) GPIO23
   3V3 (17) (18) GPIO24
GPIO10 (19) (20) GND   
 GPIO9 (21) (2